<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').config('spark.ui.port', '4050').getOrCreate()

In [6]:

sc = spark.sparkContext

In [5]:
import requests
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkFiles
import uuid

In [ ]:
import requests
from pyspark.sql.types import *

def readFromAPI(url: str, schema: StructType = None):
  response = requests.get(url)
  rdd = sc.parallelize(response.json())

  if schema:
    df = spark.read.schema(schema).json(rdd)
  else:
    df = spark.read.json(rdd)
  return df

In [ ]:
import pyspark.sql.functions as F

# read stream
stream = spark.readStream.format("rate").load()

In [10]:
from pyspark import SparkFiles
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import uuid
from pyspark.sql.functions import udf

def get_data(url: str, filename: str, schema) -> DataFrame:
  # read using SparkFiles
  spark.sparkContext.addFile(url)
  data = sc.textFile(SparkFiles.get(filename))

  # removing header
  header = data.first()
  data = data.filter(lambda line: line != header)

  df = spark.read.csv(data, header=True, schema=schema)
  return df

@udf
def generate_uuid():
  return str(uuid.uuid4())

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

df = get_data(url = "https://api.carrismetropolitana.pt/vehicles", filename=vehicles, schema = vehicle_schema)
df = df.withColumn("hash", generate_uuid())

df = df.cache()
df.count()

df.show()

NameError: name 'vehicles' is not defined

In [4]:
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])
